# 실시간 손 추적 및 로봇 손 제어 (Jupyter Notebook 버전)

이 노트북은 웹캠을 사용하여 실시간으로 사용자의 손을 추적하고, 감지된 손의 모양을 로봇 손으로 따라 하도록 제어하는 프로젝트의 통합 버전입니다.

### 주요 기능
- **실시간 손 감지**: `Mediapipe` 라이브러리를 사용하여 최대 2개의 손(왼손/오른손)을 실시간으로 감지하고 21개의 3D 랜드마크를 추출합니다.
- **손가락 각도 계산**: 추출된 랜드마크를 기반으로 각 손가락 관절의 각도를 정밀하게 계산합니다.
- **로봇 손 제어**: 계산된 각도를 서보 모터가 인식할 수 있는 신호로 변환하여, `Serial` 통신을 통해 로봇 손에 명령을 전송합니다.
- **시각화**: 감지된 손의 랜드마크, 뼈대, FPS(초당 프레임 수) 등 처리 결과를 OpenCV 창에 실시간으로 표시합니다.

### 실행 순서
1. 아래의 '환경 설정' 셀부터 순서대로 실행하세요 (`Shift + Enter`).
2. '설정값 구성' 셀에서 자신의 환경에 맞게 **`COM_PORT`** 등 주요 변수를 수정합니다.
3. '메인 실행' 셀을 실행하면 OpenCV 창이 나타나며, 실시간 영상 처리가 시작됩니다.
4. 실행을 중지하려면 **OpenCV 창을 선택하고 `ESC` 키**를 누르거나, Jupyter의 **'정지(■)' 버튼**을 눌러 커널을 중단하세요.

## 1. 환경 설정

프로젝트 실행에 필요한 라이브러리들을 가져옵니다. 만약 라이브러리가 설치되어 있지 않다면, 아래 주석 처리된 `!pip install` 명령어를 사용하여 설치하세요.

In [1]:
# !pip install opencv-python mediapipe numpy pyserial matplotlib
# !pip install mediapipe=0.10.5 < 버전 확인하기

# 필요한 라이브러리 임포트
import cv2 as cv
import numpy as np
import mediapipe as mp
import serial
import time
import math
from collections import deque
import copy
import matplotlib.pyplot as plt

## 2. FPS 계산 클래스

영상 처리 속도(FPS)를 계산하고 안정적인 평균값을 제공하는 유틸리티 클래스입니다.

In [2]:
class CvFpsCalc(object):
    """
    FPS 계산을 담당하는 클래스.
    호출될 때마다 프레임 간의 경과 시간을 측정하고,
    최근 N개의 경과 시간(difftimes)을 저장하여 평균 FPS를 계산합니다.
    """
    def __init__(self, buffer_len=10):
        """
        클래스 초기화 메서드.
        :param buffer_len: FPS를 계산할 때 평균을 낼 프레임의 수.
        """
        self._start_tick = cv.getTickCount()
        self._freq = 1000.0 / cv.getTickFrequency()
        self._difftimes = deque(maxlen=buffer_len)

    def get(self):
        """
        FPS를 계산하고 반환하는 메서드.
        메인 루프에서 새로운 프레임을 처리할 때마다 한 번씩 호출되어야 합니다.
        :return: 계산된 FPS 값 (소수점 둘째 자리까지 반올림).
        """
        current_tick = cv.getTickCount()
        different_time = (current_tick - self._start_tick) * self._freq
        self._start_tick = current_tick

        self._difftimes.append(different_time)

        # deque에 저장된 최근 시간 경과들의 평균을 구합니다.
        avg_difftime = sum(self._difftimes) / len(self._difftimes)

        # FPS를 계산합니다.
        fps = 1000.0 / avg_difftime
        fps_rounded = round(fps, 2)

        return fps_rounded

## 3. 손가락 제어 클래스

손의 랜드마크를 분석하여 각도를 계산하고, 시리얼 통신으로 로봇 손에 명령을 전송하는 핵심 클래스입니다.

In [3]:
class FingerControl:
    """
    손가락 제어를 담당하는 클래스.
    Mediapipe에서 얻은 랜드마크 좌표를 바탕으로 손가락 각도를 계산하고,
    이를 서보 모터 제어 신호로 변환하여 시리얼 통신으로 전송하는 역할을 합니다.
    """
    def __init__(self, com='COM10'):
        """
        클래스 초기화 메서드.
        """
        self.com_port_name = com
        self.serial_port = None
        self.__time_tick_right = time.time()
        self.__time_tick_left = time.time()
        self.__servo_angle_right = [0, 0, 0, 0, 0]
        self.__servo_angle_left = [0, 0, 0, 0, 0]
        
        # 시리얼 포트 연결 시도
        try:
            self.serial_port = serial.Serial(self.com_port_name, 115200, timeout=0.1)
            print(f"{self.com_port_name}에 성공적으로 연결되었습니다.")
        except serial.SerialException as e:
            print(f"오류: {self.com_port_name}에 연결할 수 없습니다. {e}")
            print("로봇 손 제어 없이 시각화만 진행됩니다.")

    def __vector_length(self, v):
        return math.sqrt(sum(x*x for x in v))

    def __dot_product(self, v1, v2):
        return sum(x*y for x, y in zip(v1, v2))

    def __angle_between_vectors(self, v1, v2):
        dot_prod = self.__dot_product(v1, v2)
        v1_length = self.__vector_length(v1)
        v2_length = self.__vector_length(v2)
        cos_theta = dot_prod / (v1_length * v2_length)
        angle_rad = math.acos(max(-1.0, min(1.0, cos_theta)))
        return math.degrees(angle_rad)

    def __finger_angle(self, p1, p2, q1, q2):
        vector_A = [(p2.x - p1.x), (p2.y - p1.y), (p2.z - p1.z)]
        vector_B = [(q2.x - q1.x), (q2.y - q1.y), (q2.z - q1.z)]
        angle = self.__angle_between_vectors(vector_A, vector_B)
        return angle
    
    def __linear_transform(self, x, a_min, a_max, b_min, b_max):
        x = max(min(x, a_max), a_min)
        return (x - a_min) * (b_max - b_min) / (a_max - a_min) + b_min
    
    def __serial_write(self, cmd):
        """시리얼 포트가 열려 있을 경우에만 데이터를 전송합니다."""
        if self.serial_port and self.serial_port.is_open:
            self.serial_port.write(cmd.encode())

    def __serial_write_right(self, servo_angle):
        cmd = "FR0%03d1%03d2%03d3%03d4%03d" % (servo_angle[0], servo_angle[1], servo_angle[2], servo_angle[3], servo_angle[4])
        self.__serial_write(cmd)

    def __serial_write_left(self, servo_angle):
        cmd = "FL5%03d6%03d7%03d8%03d9%03d" % (servo_angle[0], servo_angle[1], servo_angle[2], servo_angle[3], servo_angle[4])
        self.__serial_write(cmd)

    def finger_robot_right(self, world_landmarks):
        world_angle = [0, 0, 0, 0, 0]
        servo_angle = [0, 0, 0, 0, 0]

        world_angle[0] = self.__finger_angle(world_landmarks.landmark[2], world_landmarks.landmark[3], world_landmarks.landmark[2], world_landmarks.landmark[17])
        world_angle[1] = self.__finger_angle(world_landmarks.landmark[5], world_landmarks.landmark[8], world_landmarks.landmark[5], world_landmarks.landmark[0])
        world_angle[2] = self.__finger_angle(world_landmarks.landmark[9], world_landmarks.landmark[12], world_landmarks.landmark[9], world_landmarks.landmark[0])
        world_angle[3] = self.__finger_angle(world_landmarks.landmark[13], world_landmarks.landmark[16], world_landmarks.landmark[13], world_landmarks.landmark[0])
        world_angle[4] = self.__finger_angle(world_landmarks.landmark[17], world_landmarks.landmark[20], world_landmarks.landmark[17], world_landmarks.landmark[0])

        servo_angle[0] = self.__linear_transform(world_angle[0], 60, 120, 90, 0)
        servo_angle[1] = self.__linear_transform(world_angle[1], 30, 160, 180, 0)
        servo_angle[2] = self.__linear_transform(world_angle[2], 30, 160, 0, 180)
        servo_angle[3] = self.__linear_transform(world_angle[3], 30, 160, 180, 0)
        servo_angle[4] = self.__linear_transform(world_angle[4], 30, 160, 0, 180)

        time_now = time.time()
        bSendSerial = False
        for index in range(5):
            if abs(self.__servo_angle_right[index] - servo_angle[index]) >= 30:
                bSendSerial = True
                break
        
        if (time_now - self.__time_tick_right) >= 0.03 and bSendSerial:
            self.__time_tick_right = time_now
            self.__serial_write_right(servo_angle)
            self.__servo_angle_right = servo_angle
        
        return world_angle, servo_angle

    def finger_robot_left(self, world_landmarks):
        world_angle = [0, 0, 0, 0, 0]
        servo_angle = [0, 0, 0, 0, 0]

        world_angle[0] = self.__finger_angle(world_landmarks.landmark[2], world_landmarks.landmark[3], world_landmarks.landmark[2], world_landmarks.landmark[17])
        world_angle[1] = self.__finger_angle(world_landmarks.landmark[5], world_landmarks.landmark[8], world_landmarks.landmark[5], world_landmarks.landmark[0])
        world_angle[2] = self.__finger_angle(world_landmarks.landmark[9], world_landmarks.landmark[12], world_landmarks.landmark[9], world_landmarks.landmark[0])
        world_angle[3] = self.__finger_angle(world_landmarks.landmark[13], world_landmarks.landmark[16], world_landmarks.landmark[13], world_landmarks.landmark[0])
        world_angle[4] = self.__finger_angle(world_landmarks.landmark[17], world_landmarks.landmark[20], world_landmarks.landmark[17], world_landmarks.landmark[0])

        servo_angle[0] = self.__linear_transform(world_angle[0], 60, 110, 180, 0)
        servo_angle[1] = self.__linear_transform(world_angle[1], 30, 160, 0, 180)
        servo_angle[2] = self.__linear_transform(world_angle[2], 30, 160, 0, 180)
        servo_angle[3] = self.__linear_transform(world_angle[3], 30, 160, 180, 0)
        servo_angle[4] = self.__linear_transform(world_angle[4], 30, 160, 0, 180)

        time_now = time.time()
        bSendSerial = False
        for index in range(5):
            if abs(self.__servo_angle_left[index] - servo_angle[index]) >= 25:
                bSendSerial = True
                break

        if (time_now - self.__time_tick_left) >= 0.03 and bSendSerial:
            self.__time_tick_left = time_now
            self.__serial_write_left(servo_angle)
            self.__servo_angle_left = servo_angle
        
        return world_angle, servo_angle
    
    def close(self):
        """시리얼 포트를 닫습니다."""
        if self.serial_port and self.serial_port.is_open:
            self.serial_port.close()
            print(f"{self.com_port_name} 연결을 해제했습니다.")

## 4. 시각화 헬퍼 함수

OpenCV 이미지 위에 랜드마크, 경계 상자, 텍스트 등을 그리거나, Matplotlib으로 3D 랜드마크를 플로팅하는 함수들입니다.

In [4]:
def calc_palm_moment(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    palm_array = np.empty((0, 2), int)
    for index, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point = [np.array((landmark_x, landmark_y))]
        if index in [0, 1, 5, 9, 13, 17]:
            palm_array = np.append(palm_array, landmark_point, axis=0)
    M = cv.moments(palm_array)
    cx, cy = 0, 0
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    return cx, cy

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point = [np.array((landmark_x, landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)
    x, y, w, h = cv.boundingRect(landmark_array)
    return [x, y, x + w, y + h]

def draw_landmarks(image, cx, cy, landmarks, handedness):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_point = []
    for index, landmark in enumerate(landmarks.landmark):
        if landmark.visibility < 0 or landmark.presence < 0:
            continue
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point.append((landmark_x, landmark_y))
        cv.circle(image, (landmark_x, landmark_y), 5, (0, 255, 0), 1)
        if index in [4, 8, 12, 16, 20]:
            cv.circle(image, (landmark_x, landmark_y), 12, (0, 255, 0), 1)

    if len(landmark_point) > 0:
        # 뼈대 그리기 (생략 - 원본 코드 참조)
        # 엄지
        cv.line(image, landmark_point[ 2], landmark_point[ 3], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 3], landmark_point[ 4], (0, 255, 0), 1)
        # 검지
        cv.line(image, landmark_point[ 5], landmark_point[ 6], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 6], landmark_point[ 7], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 7], landmark_point[ 8], (0, 255, 0), 1)
        # 중지
        cv.line(image, landmark_point[ 9], landmark_point[10], (0, 255, 0), 1)
        cv.line(image, landmark_point[10], landmark_point[11], (0, 255, 0), 1)
        cv.line(image, landmark_point[11], landmark_point[12], (0, 255, 0), 1)
        # 약지
        cv.line(image, landmark_point[13], landmark_point[14], (0, 255, 0), 1)
        cv.line(image, landmark_point[14], landmark_point[15], (0, 255, 0), 1)
        cv.line(image, landmark_point[15], landmark_point[16], (0, 255, 0), 1)
        # 소지
        cv.line(image, landmark_point[17], landmark_point[18], (0, 255, 0), 1)
        cv.line(image, landmark_point[18], landmark_point[19], (0, 255, 0), 1)
        cv.line(image, landmark_point[19], landmark_point[20], (0, 255, 0), 1)
        # 손바닥
        cv.line(image, landmark_point[ 0], landmark_point[ 1], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 1], landmark_point[ 2], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 2], landmark_point[ 5], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 5], landmark_point[ 9], (0, 255, 0), 1)
        cv.line(image, landmark_point[ 9], landmark_point[13], (0, 255, 0), 1)
        cv.line(image, landmark_point[13], landmark_point[17], (0, 255, 0), 1)
        cv.line(image, landmark_point[17], landmark_point[ 0], (0, 255, 0), 1)

    if len(landmark_point) > 0:
        cv.circle(image, (cx, cy), 12, (0, 0, 255), 1)
        cv.putText(image, handedness.classification[0].label[0], (cx - 6, cy + 6), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, cv.LINE_AA)
    return image

def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]), (0, 255, 0), 2)
    return image

def plot_world_landmarks(plt, ax_list, multi_hands_landmarks, multi_handedness):
    ax_list[0].cla()
    ax_list[0].set_xlim3d(-0.1, 0.1)
    ax_list[0].set_ylim3d(-0.1, 0.1)
    ax_list[0].set_zlim3d(-0.1, 0.1)
    ax_list[1].cla()
    ax_list[1].set_xlim3d(-0.1, 0.1)
    ax_list[1].set_ylim3d(-0.1, 0.1)
    ax_list[1].set_zlim3d(-0.1, 0.1)

    for landmarks, handedness in zip(multi_hands_landmarks, multi_handedness):
        handedness_index = 0
        if handedness.classification[0].label == 'Left': handedness_index = 0
        elif handedness.classification[0].label == 'Right': handedness_index = 1

        landmark_point = [[lm.visibility, (lm.x, lm.y, lm.z)] for lm in landmarks.landmark]
        
        palm_list = [0, 1, 5, 9, 13, 17, 0]
        thumb_list = [1, 2, 3, 4]
        index_finger_list = [5, 6, 7, 8]
        middle_finger_list = [9, 10, 11, 12]
        ring_finger_list = [13, 14, 15, 16]
        pinky_list = [17, 18, 19, 20]

        def extract_points(part_list):
            x, y, z = [], [], []
            for index in part_list:
                point = landmark_point[index][1]
                x.append(point[0])
                y.append(point[2])
                z.append(point[1] * (-1))
            return x, y, z

        ax_list[handedness_index].plot(*extract_points(palm_list))
        ax_list[handedness_index].plot(*extract_points(thumb_list))
        ax_list[handedness_index].plot(*extract_points(index_finger_list))
        ax_list[handedness_index].plot(*extract_points(middle_finger_list))
        ax_list[handedness_index].plot(*extract_points(ring_finger_list))
        ax_list[handedness_index].plot(*extract_points(pinky_list))

    plt.pause(.001)

## 5. 설정값 구성

이곳에서 프로그램의 주요 설정들을 변경할 수 있습니다. **특히 `COM_PORT`는 자신의 로봇 손이 연결된 포트 번호로 반드시 수정해야 합니다.**

In [5]:
# ----- 주요 설정값 ----- 

# 시리얼 포트 설정 (Windows: 'COMx', Linux: '/dev/ttyACMx' 또는 '/dev/ttyUSBx')
COM_PORT = 'COM10' 

# 카메라 설정
CAP_DEVICE = 0
CAP_WIDTH = 960
CAP_HEIGHT = 540

# MediaPipe Hands 모델 설정
MODEL_COMPLEXITY = 1
MAX_NUM_HANDS = 2
MIN_DETECTION_CONFIDENCE = 0.7
MIN_TRACKING_CONFIDENCE = 0.5

# 시각화 설정
USE_BRECT = True  # 손 주위에 경계 사각형 그리기
PLOT_WORLD_LANDMARK = False # 3D 랜드마크를 실시간으로 플로팅 (성능 저하 가능성)

## 6. 메인 실행

아래 셀을 실행하면 웹캠이 켜지고 실시간 손 추적이 시작됩니다.

**- 종료 방법:** 실행 후 나타나는 **'Hell Hand' 창을 클릭하고 `ESC` 키**를 누르세요.
- 만약 창이 반응하지 않으면, 이 노트북 상단의 **'정지(■)' 버튼**을 눌러 셀 실행을 강제로 중단할 수 있습니다.

In [6]:
# -------------------------
# 메인 실행 로직
# -------------------------

# FPS 계산기 초기화
cvFpsCalc = CvFpsCalc(buffer_len=10)

# 손가락 제어 클래스 초기화
finger_controller = FingerControl(com=COM_PORT)

# 카메라 준비
cap = cv.VideoCapture(CAP_DEVICE, cv.CAP_DSHOW)
cap.set(cv.CAP_PROP_FRAME_WIDTH, CAP_WIDTH)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, CAP_HEIGHT)

# MediaPipe Hands 모델 로드
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    model_complexity=MODEL_COMPLEXITY,
    max_num_hands=MAX_NUM_HANDS,
    min_detection_confidence=MIN_DETECTION_CONFIDENCE,
    min_tracking_confidence=MIN_TRACKING_CONFIDENCE,
    )

# 3D 플롯 준비 (옵션)
if PLOT_WORLD_LANDMARK:
    fig = plt.figure()
    r_ax = fig.add_subplot(121, projection="3d")
    l_ax = fig.add_subplot(122, projection="3d")
    fig.subplots_adjust(left=0.0, right=1, bottom=0, top=1)
else:
    fig, r_ax, l_ax = None, None, None

try:
    while True:
        display_fps = cvFpsCalc.get()
        ret, image = cap.read()
        if not ret:
            print("카메라 프레임을 읽을 수 없습니다. 종료합니다.")
            break

        image = cv.flip(image, 1)
        debug_image = copy.deepcopy(image)

        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                cx, cy = calc_palm_moment(debug_image, hand_landmarks)
                brect = calc_bounding_rect(debug_image, hand_landmarks)
                debug_image = draw_landmarks(debug_image, cx, cy, hand_landmarks, handedness)
                debug_image = draw_bounding_rect(USE_BRECT, debug_image, brect)

        cv.putText(debug_image, "FPS:" + str(display_fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2, cv.LINE_AA)

        if results.multi_hand_world_landmarks is not None:
            if PLOT_WORLD_LANDMARK:
                plot_world_landmarks(plt, [r_ax, l_ax], results.multi_hand_world_landmarks, results.multi_handedness)

            work_left = False
            work_right = False
            for landmarks, handedness in zip(results.multi_hand_world_landmarks, results.multi_handedness):
                if handedness.classification[0].label == 'Left' and not work_left:
                    work_left = True
                    finger_controller.finger_robot_left(landmarks)
                elif handedness.classification[0].label == 'Right' and not work_right:
                    work_right = True
                    finger_controller.finger_robot_right(landmarks)

        key = cv.waitKey(1)
        if key == 27:  # ESC 키
            print("ESC 키가 입력되어 프로그램을 종료합니다.")
            break

        cv.imshow('Hell Hand - Jupyter', debug_image)

finally:
    # 프로그램 종료 시 자원 해제
    print("자원을 해제합니다.")
    cap.release()
    cv.destroyAllWindows()
    finger_controller.close() # 시리얼 포트 닫기
    # Matplotlib 창 닫기 (오류 방지)
    if PLOT_WORLD_LANDMARK:
        plt.close('all')

COM10에 성공적으로 연결되었습니다.
ESC 키가 입력되어 프로그램을 종료합니다.
자원을 해제합니다.
COM10 연결을 해제했습니다.
